In [1]:
import numpy as np
from numpy.typing import NDArray
from pathlib import Path

from tarea2.algorithms import simplex_normalization, north_west_heuristic, costos_reducidos
from tarea2.io import get_data
from tarea2.gen import gen_A , gen_b, gen_mapping


# Obtener los datos

In [2]:
A = np.array([
  [1, -2, 1, 0],
  [-1, 1, 0, 1]
], dtype=np.float64)

b = np.array([
  [4],
  [3]
])

c = np.array([
  [-1],
  [-3],
  [0],
  [0],
])

B = np.array([2, 3])
NB = np.array([0, 1])

In [3]:
A_B = A[:, B]
A_NB = A[:, NB]

c_B = c[B]
c_NB = c[NB]

In [4]:
X = np.zeros(shape=c.shape)

X[B] = np.dot(np.linalg.inv(A[:,B]) ,b)
X

array([[0.],
       [0.],
       [4.],
       [3.]])

Todos los elementos de la solución base sean positivos para ser **solución básica factible**

In [5]:
from dataclasses import dataclass

@dataclass
class direction:
  NB_i: int
  d: NDArray
  cr: NDArray

def direction_and_reduced_cost(X: NDArray, A: NDArray, B:NDArray, NB: NDArray, NB_i: int) -> NDArray:
  d = np.zeros(shape=X.shape)
  d_B = - np.dot(
    np.linalg.inv(A[:,B]),
    A[:, NB_i]
  )

  np.put(d, NB_i, 1)
  np.put(d, B, d_B)


  cr = c[NB_i] + np.dot(c[B].T, d[B])

  return direction( NB_i, d, cr.flatten())


d_cr_list = list(map(lambda i: direction_and_reduced_cost(X, A, B, NB, i), NB))

print(d_cr_list)

[direction(NB_i=0, d=array([[ 1.],
       [ 0.],
       [-1.],
       [ 1.]]), cr=array([-1.])), direction(NB_i=1, d=array([[ 0.],
       [ 1.],
       [ 2.],
       [-1.]]), cr=array([-3.]))]


In [6]:
# Determinar dirección con menor costo reducido

d_cr_min: direction = min(d_cr_list, key=lambda x: x.cr)

d_cr_min
NB_entra = d_cr_min.NB_i

In [7]:
# Obtener componentes de la dirección de menor costo reducido
# que sean de la base y menores a cero

# Lista de índices de la base en que el componente de la distancia sea menor a cero.
B_menores_a_cero = [B[i] for i, d_i in enumerate( d_cr_min.d[B] ) if d_i < 0]

print(f"{B_menores_a_cero = }") # d[3] = [0 1 2 -1]

# Dos pasos
# 1. Obtener la lista de desplazamientos posibles
# 2. Quedarse con el menor phi, y determinar su índice (El que sale)
phi_list = [(- X[i] / d_cr_min.d[i], i) for i in B_menores_a_cero]

phi, B_sale = min(phi_list)

B_sale_row = np.where(B == B_sale)[0][0]

print(f"{phi = }")
print(B)
print(f"{B_sale = }")
print(f"{B_sale_row = }")



B_menores_a_cero = [3]
phi = array([3.])
[2 3]
B_sale = 3
B_sale_row = 1


In [8]:
print(f"Entra: {NB_entra}")
print(f"Sale: {B_sale}")

Entra: 1
Sale: 3


In [9]:
# Usamos el algoritmo de 
A_2 = simplex_normalization(A=A, entra=NB_entra, sale=B_sale_row)
A_2

array([[-1.,  0.,  1.,  2.],
       [-1.,  1.,  0.,  1.]])

In [10]:
# Usamos el algoritmo de 
A_2 = simplex_normalization(A=A, entra=NB_entra, sale=B_sale_row)
A_2

array([[-1.,  0.,  1.,  2.],
       [-1.,  1.,  0.,  1.]])

Para ser solución base, el producto con la inversa de A_B debe ser b
$$
A_B x_B = b
\\
 x_B = A_B^{-1} b
$$

# Costos Reducidos

In [11]:
cr = costos_reducidos(B, NB, A, c)

def es_factible(X_B):
  return (X_B >= 0).all()

def es_optimo(cr):

  if (cr >= 0).all():
    print("Tenemos un óptimo")

    if (cr == 0).any():
      print("Son múltiples óptimos")
    else:
      print("Es único")

    return True

  return False


if (es_factible(X_B)):
  es_optimo(cr)

ValueError: shapes (2,1) and (2,2) not aligned: 1 (dim 1) != 2 (dim 0)

Obtener la dirección básica N-ésima

In [ ]:
d = np.zeros(shape=X.shape)
d_B = - np.dot( np.linalg.inv(A_B), A[:, NB[1]] )
#d_NB = d[idx_NB]

#d_B[:] = 1
#d.sum()



d_B